In [1]:
import numpy as np

def sigmoid(Z):


    A = 1/(1+np.exp(-Z))
    cache = Z

    return A, cache

def relu(Z):


    A = np.maximum(0,Z)

    assert(A.shape == Z.shape)

    cache = Z
    return A, cache


def relu_backward(dA, cache):


    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.

    # When z <= 0, you should set dz to 0 as well.
    dZ[Z <= 0] = 0

    assert (dZ.shape == Z.shape)

    return dZ

def sigmoid_backward(dA, cache):


    Z = cache

    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)

    assert (dZ.shape == Z.shape)

    return dZ



In [2]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

import copy
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

In [3]:
def initialize_parameters(n_x, n_h, n_y):


    np.random.seed(1)


    # YOUR CODE STARTS HERE

    W1 = np.random.randn(n_h,n_x) * 0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h) * 0.01
    b2 = np.zeros((n_y,1))


    # YOUR CODE ENDS HERE

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}

    return parameters

In [5]:
def initialize_parameters_deep(layer_dims):


    np.random.seed(3)
    parameters = {}
    L = len(layer_dims) # number of layers in the network

    for l in range(1, L):

        # YOUR CODE STARTS HERE
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        # YOUR CODE ENDS HERE

        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l - 1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))


    return parameters

In [7]:
testA = np.array([[1,2,3],[2,3,4]])
print(testA)
print(testA.shape)

[[1 2 3]
 [2 3 4]]
(2, 3)


In [10]:
assert (testA.shape == (2,3))


In [11]:

def linear_forward(A, W, b):



    # YOUR CODE STARTS HERE
    Z = np.dot(W,A) + b

    # YOUR CODE ENDS HERE
    cache = (A, W, b)

    return Z, cache

In [12]:
def linear_activation_forward(A_prev, W, b, activation):


    if activation == "sigmoid":

        # YOUR CODE STARTS HERE
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
        # YOUR CODE ENDS HERE

    elif activation == "relu":

        # YOUR CODE STARTS HERE
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)

        # YOUR CODE ENDS HERE
    cache = (linear_cache, activation_cache)

    return A, cache

In [14]:
def L_model_forward(X, parameters):


    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network

    # The for loop starts at 1 because layer 0 is the input
    for l in range(1, L):
        A_prev = A

        # YOUR CODE STARTS HERE
        A, cache = linear_activation_forward(A_prev, parameters["W"+str(l)], parameters["b"+str(l)], activation = "relu")
        caches.append(cache)
        # YOUR CODE ENDS HERE


    # YOUR CODE STARTS HERE
    AL, cache = linear_activation_forward(A, parameters["W"+str(L)], parameters["b"+str(L)], activation = "sigmoid")
    caches.append(cache)
    # YOUR CODE ENDS HERE

    return AL, caches

In [15]:
def compute_cost(AL, Y):


    m = Y.shape[1]


    # YOUR CODE STARTS HERE
    cost = (np.dot(Y,np.log(AL.T)) + np.dot((1-Y),np.log(1-AL.T)))/(-m)

    # YOUR CODE ENDS HERE

    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).


    return cost

In [18]:
A = np.array([[1, 2], [3, 4]])
A

array([[1, 2],
       [3, 4]])

In [20]:
A = np.array([[1, 2], [3, 4]])

print('axis=1 and keepdims=True')
print(np.sum(A, axis=1, keepdims=True))
print('axis=1 and keepdims=False')
print(np.sum(A, axis=1, keepdims=False))
print('axis=0 and keepdims=True')
print(np.sum(A, axis=0, keepdims=True))
print('axis=0 and keepdims=False')
print(np.sum(A, axis=0, keepdims=False))

axis=1 and keepdims=True
[[3]
 [7]]
axis=1 and keepdims=False
[3 7]
axis=0 and keepdims=True
[[4 6]]
axis=0 and keepdims=False
[4 6]


In [21]:
def linear_backward(dZ, cache):

    A_prev, W, b = cache
    m = A_prev.shape[1]


    # YOUR CODE STARTS HERE
    dW = np.dot(dZ,A_prev.T)/m
    db = np.sum(dZ, axis=1,keepdims=True)/m
    dA_prev = np.dot(W.T,dZ)

    # YOUR CODE ENDS HERE

    return dA_prev, dW, db

In [22]:
def linear_activation_backward(dA, cache, activation):

    linear_cache, activation_cache = cache

    if activation == "relu":

        # YOUR CODE STARTS HERE
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        # YOUR CODE ENDS HERE

    elif activation == "sigmoid":

        # YOUR CODE STARTS HERE
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        # YOUR CODE ENDS HERE

    return dA_prev, dW, db

In [23]:
def L_model_backward(AL, Y, caches):

    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL


    # YOUR CODE STARTS HERE
    dAL =  - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

    # YOUR CODE ENDS HERE


    # YOUR CODE STARTS HERE
    current_cache = caches[L-1]
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dAL, current_cache, activation="sigmoid")
    grads["dA" + str(L-1)] = dA_prev_temp
    grads["dW" + str(L)] = dW_temp
    grads["db" + str(L)] = db_temp
    # YOUR CODE ENDS HERE

    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):

        # YOUR CODE STARTS HERE
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation="relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l+1)] = dW_temp
        grads["db" + str(l+1)] = db_temp

        # YOUR CODE ENDS HERE

    return grads

In [24]:
def update_parameters(params, grads, learning_rate):

    parameters = copy.deepcopy(params)
    L = len(parameters) // 2 # number of layers in the neural network

    #(≈ 2 lines of code)
    for l in range(L):

        # YOUR CODE STARTS HERE
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - (learning_rate * grads["dW" + str(l+1)])
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - (learning_rate * grads["db" + str(l+1)])
        # YOUR CODE ENDS HERE
    return parameters